In [6]:
import os
import requests
from bs4 import BeautifulSoup
import re
import csv

In [9]:
def retrieve_cells(csv_file):
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        data = list(reader)[1:]  # Skip the first row

        cells = []
        for row in data[:4]:  # Retrieve 4 rows
            row_cells = row[2:12]  # Retrieve 10 cells, skipping first two columns
            cells.append(row_cells)

        return cells

# Example usage:
csv_file = '/home/pierluigi/Downloads/Text Analytics - 02_05_2023.csv'  # Replace with the path to your CSV file
retrieved_cells = retrieve_cells(csv_file)

# Print the retrieved cells
for row in retrieved_cells:
    print(row)


['https://news.google.com/articles/CBMiTGh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjMvMDUvMDIvb3Bpbmlvbi9yb24tZGVzYW50aXMtd29rZS1taW5kLXZpcnVzLmh0bWzSAQA?hl=en-US&gl=US&ceid=US%3Aen', 'https://news.google.com/articles/CBMieGh0dHBzOi8vd3d3LmJpbGxib2FyZC5jb20vY3VsdHVyZS90di1maWxtL2FkYW0tbGFtYmVydC1wZXJmb3JtYW5jZS1hbWVyaWNhbi1pZG9sLWNvdmVyLWktY2FudC1zdGFuZC10aGUtcmFpbi0xMjM1MzE4ODc2L9IBfGh0dHBzOi8vd3d3LmJpbGxib2FyZC5jb20vY3VsdHVyZS90di1maWxtL2FkYW0tbGFtYmVydC1wZXJmb3JtYW5jZS1hbWVyaWNhbi1pZG9sLWNvdmVyLWktY2FudC1zdGFuZC10aGUtcmFpbi0xMjM1MzE4ODc2L2FtcC8?hl=en-US&gl=US&ceid=US%3Aen', 'https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LnRoZS1zdW4uY29tL2VudGVydGFpbm1lbnQvODAwNzY3Mi9hbWVyaWNhbi1pZG9sLWZhbnMtYmVnLXNob3ctaGlyZS1hbHVtLWFkYW0tbGFtYmVydC_SAWVodHRwczovL3d3dy50aGUtc3VuLmNvbS9lbnRlcnRhaW5tZW50LzgwMDc2NzIvYW1lcmljYW4taWRvbC1mYW5zLWJlZy1zaG93LWhpcmUtYWx1bS1hZGFtLWxhbWJlcnQvYW1wLw?hl=en-US&gl=US&ceid=US%3Aen', 'https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3Lm1qc2JpZ2Jsb2cuY29tL3dhdGNoLW

In [ ]:
def save_article_text(url):
    # Set headers to mimic a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Send a GET request to the URL with headers
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the title, subtitle, description, and main text
    title_element = soup.find('title')
    title = title_element.text.strip() if title_element else ""

    subtitle_element = soup.find('meta', attrs={'name': 'description'})
    subtitle = subtitle_element['content'].strip() if subtitle_element and 'content' in subtitle_element.attrs else ""

    description_element = soup.find('meta', attrs={'name': 'og:description'})
    description = description_element['content'].strip() if description_element and 'content' in description_element.attrs else ""

    # Find and exclude unwanted elements by class names or content patterns
    unwanted_elements = soup.find_all(['script', 'style', 'a', 'div', 'span'], class_=['follow-us', 'newsletter', 'advertisement'])
    patterns_to_exclude = ['next article', 'read next', 'correlated']
    for element in unwanted_elements:
        if any(pattern in str(element).lower() for pattern in patterns_to_exclude):
            element.extract()

    # Find and exclude footer container and "All rights reserved" text
    footer_elements = soup.find_all(['footer', 'div'], class_=['footer', 'bottom-footer'])
    for element in footer_elements:
        element.extract()
    all_rights_reserved_elements = soup.find_all(text=re.compile(r'\bAll rights reserved\b', re.IGNORECASE))
    for element in all_rights_reserved_elements:
        element.extract()

    # Find the main text element(s) based on the HTML structure of the page
    main_text_elements = soup.find_all('p')
    main_text = "\n\n".join([element.text.strip() for element in main_text_elements if element.text.strip()])

    # Set the subtitle to the description if it is empty
    if not subtitle:
        subtitle = description.strip()

    # Concatenate the extracted strings
    article_text = f"{title}\n\n{subtitle}\n\n{main_text}"

    return article_text


In [ ]:
def create_csv_file(urls):
    # Create a CSV file named "articles.csv"
    filename = "articles.csv"

    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)

        # Iterate over the URLs
        for i, url in enumerate(urls, start=1):
            try:
                # Call save_article_text to get the article text
                article_text = save_article_text(url)

                # Write the article text to the CSV file
                writer.writerow([article_text])

                # Start a new row after every 10 URLs
                if i % 10 == 0:
                    writer.writerow([])  # Add an empty row

                print(f"Article processed: {url}")
            except (requests.exceptions.RequestException, requests.exceptions.HTTPError) as e:
                # Handle the exception and continue with the next URL
                print(f"Error processing URL: {url}")
                print(f"Error message: {str(e)}")
                continue

    print(f"CSV file created: {filename}")